# Deploy Model with either ACI or AKS
Now it's time to deploy your model.  There are many ways to deploy models in Azure.  For testing purposes, we recommend Azure Container Instance (ACI).  For production purposes, we recommend Azure Kubernetes Service (AKS).  AKS requires a cluster to run on, but has much more robust security options and can run inside of a virtual network.<br>

Both AKS and ACI are used for real-time or near real-time scoring situations.  These are situations where you want to send data to the system and immediately get a result to use in some process.

To learn more about deployment, click here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where



In [24]:
# Load Azure Libaries
from azureml.core import Datastore
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azure.storage.blob import BlockBlobService
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.explain.model._internal.explanation_client import ExplanationClient

#Load Libraries for Deployment
from azureml.core.model import Model
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget

# Import other Libraries
import pandas as pd
import numpy as np
import logging
import requests
import json
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Retrieve your workspace by name by filling in the lower case values between double quotes
ws = Workspace.get(name="ancient-rivers-ml-workspace",
        subscription_id="47a7ec0c-37ad-428b-9114-b87ea1057632",
        resource_group="xeek-ancient-rivers")

In [3]:
# Retrieve your Datastore by name by filling in the lower case values between double quotes
datastore_name = "ancientrivers"
datastore = Datastore.get(ws, datastore_name)

In [4]:
# Retrieve your Datasets by name by filling in the lower case values between double quotes
dataset_name_test  = "ancient-rivers-test-transformed"
dataset_name_train = "ancient-rivers-train-transformed"

# Load Data in as Tabular Datasets
testing_data  = Dataset.get_by_name(ws, dataset_name_test, version='latest')
training_data = Dataset.get_by_name(ws, dataset_name_train, version='latest')

In [5]:
# Convert your tabular dataset to pandas data frames
testTransformedDF = testing_data.to_pandas_dataframe()
trainTransformedDF = training_data.to_pandas_dataframe()

### Now, deploy your model to your ACI Cluster.
To deploy your model, you will need to use an entry script and an environment file.<br>
ACI is for testing and has less security, less power and shorter timeouts than AKS.

In [9]:
# Retrieve your AutoML generated entry script
entry_script = 'inference/score.py'

# Retrieve your AutoML generated environment
environment = Environment.get(ws, 'automl-environment')

# Retrieve your AutoML Model
model = Model(ws, 'AutoML37a31f86512')

In [11]:
# Give your ACI deployment a name
aci_service_name = 'automl-ancient-rivers-aci'

# Set your scoring script and environment file
inference_config = InferenceConfig(entry_script=entry_script, environment = environment)

# Set your ACI configuration options
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'Project': 'Ancient Rivers', 'Type': 'Classification'}, 
                                               description = 'Ancient Rivers Deployment')

# Deploy your model to ACI
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config,overwrite=True)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-ancient-rivers-aci
Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


### Create an AKS Cluster
Select a VM Size and number of VMs to run your AKS cluster.  Your cores must number at least 12, so if you select a VM with 4 cores, set agent count to 3.<br>
A single AKS Cluster can support many deployments depending on the size of the cluster.

Check out AKS configuration options here:<br> https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.aksprovisioningconfiguration?view=azure-ml-py

In [12]:
from azureml.core.compute import AksCompute, ComputeTarget

# Run this cell only once.
# Set the configurations here.  Select Larger VM sizes and more agents for faster performance
prov_config = AksCompute.provisioning_configuration(vm_size='Standard_D3_v2', agent_count=3)
# Give your AKS Cluster a name
aks_name = 'dms-aks-cluster'
# Create the cluster and attach it to your workspace
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

# You only need to run this cell once to set up your AKS Cluster.
# After setting it up once, hashtag this cell out unless you want to create more clusters.

Creating..........................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


### Deploy your model to your AKS Cluster.
To deploy your model, you will need to use an entry script and an environment file.<br>
AKS is for production jobs and can be configured to use large VMs, more security and longer timeouts than ACI.

In [13]:
# Retrieve your AutoML generated entry script
entry_script = 'inference/score.py'

# Retrieve your AutoML generated environment
environment = Environment.get(ws, 'automl-environment')

# Retrieve your AutoML Model
model = Model(ws, 'AutoML37a31f86512')

In [14]:
# Set your target AKS Cluster
aks_target = AksCompute(ws,'dms-aks-cluster')

# Give your deployment a name
aks_service_name = 'automl-ancient-rivers-aks'

# Set your scoring script and environment file
inference_config = InferenceConfig(entry_script=entry_script, environment = environment)

# Set your AKS configuration options
aks_config = AksWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1, 
                                                tags = {'Project': 'Ancient Rivers', 'Type': 'Classification'}, 
                                                description = 'Ancient Rivers Deployment')

# Deploy your model to AKS
print(aks_service_name)
aks_service = Model.deploy(ws, aks_service_name, [model], inference_config, aks_config, aks_target,overwrite=True)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

automl-ancient-rivers-aks
Running..........................
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


# Test your Deployment
Test your AKS and ACI deployed models through your Jupyter Notebook.  You can also use CURL (Postman).

In [33]:
# Drop columns that are not used in your scoring script
# Create a sample to display the data easily

testSampleDF = testTransformedDF[(testTransformedDF.WellID == 5001)]
testScoreDF = testSampleDF.drop(['WellID'],axis=1)


In [34]:
# Use orient records to create your JSON file.  Other formats may not work.
# Preface your autoML test data with {"data": and close it with }

testJsonLoad = '{"data":' + testScoreDF.to_json(orient='records') + '}'

In [35]:
# Load the data into JSON 
testJson = json.loads(testJsonLoad)

# Convert your test data to a JSON string
input_data = json.dumps(testJson)

### Examine your Scoring Script
Make sure that you only include variables in your test data that appear in your scoring script's inference schema.<br>  Print out your script using the cell below and compare your schema with the print statement 2 cells down.

In [36]:
#Make variables to identify where your scoring file and folders are kept
scripts_folder = "inference"
script_file = "score.py"

# peek at contents
with open(os.path.join(scripts_folder, script_file)) as inference_file:
    print(inference_file.read())

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame(data=[{'RowID': 0, 'GR': 99.0, 'smoothedGR': 104.0, 'FirstRow': 'First', 'smoothedGR1': None, 'smoothedGR-1': 104.0, 'smoothedGR2': None, 'smoothedGR-2': 104.1, 'smoothedGR3': None, 'smoothedGR-3': 104.1, 'smoothedGR4': None, 'smoothedGR-4': 104.2, 'smoothedDifference': 0.0, 'smoothedDifference_SMA': -2.8, 'smoothedDifference_SMA_4': None, 'smoothedDifference_SMA_8': None,

In [37]:
testJson

{'data': [{'RowID': 0,
   'GR': 110.5,
   'smoothedGR': 108.1,
   'FirstRow': 'First',
   'smoothedGR1': None,
   'smoothedGR-1': 108.1,
   'smoothedGR2': None,
   'smoothedGR-2': 108.1,
   'smoothedGR3': None,
   'smoothedGR-3': 108.1,
   'smoothedGR4': None,
   'smoothedGR-4': 108.1,
   'smoothedDifference': 0.0,
   'smoothedDifference_SMA': 0.0,
   'smoothedDifference_SMA_4': None,
   'smoothedDifference_SMA_8': None,
   'smoothedDifference_SMA_12': None,
   'smoothedDifference_SMA_16': None,
   'smoothedDifference_SMA_20': None,
   'smoothedDifference_SMA_24': None,
   'smoothedDifference_SMA_28': None,
   'smoothedDifference_SMA_32': None,
   'smoothedDifference_SMA_36': None,
   'smoothedDifference_SMA_40': None,
   'smoothedDifference_SMA_FORWARD_4': 0.0,
   'smoothedDifference_SMA_FORWARD_8': 0.0,
   'smoothedDifference_SMA_FORWARD_12': 0.0,
   'smoothedDifference_SMA_FORWARD_16': 0.0,
   'smoothedDifference_SMA_FORWARD_20': 0.0,
   'smoothedDifference_SMA_FORWARD_24': 0.0,
   

## Test ACI

In [38]:
# Plug in the URL for your ACI web service.  You can find it easily on ML Studio by clicking endpoints.
scoring_url_aci = 'http://4523f38e-6c26-4f79-aaa1-076c0b5b2913.eastus.azurecontainer.io/score'

# Set the content type to 'application/json'
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_url_aci, input_data, headers=headers)
print(resp.text)

"{\"result\": [\"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Lay

## Test AKS

In [39]:
# Plug in the URL for your AKS web service.  You can find it easily on ML Studio by clicking endpoints.
scoring_uri_aks = 'http://40.117.194.42:80/api/v1/service/automl-ancient-rivers-aks/score'
# Since your AKS service requires authentication, set the key or token
key = '5LYMIt2ZdZiafN2TjZcUQSlhDNXegGYB'

# Set the content type to 'application/json'
headers = {'Content-Type': 'application/json'}
# Set the authorization header for AKS
headers['Authorization'] = f'Bearer {key}'

# Make the request to your deployed model and display the response
resp = requests.post(scoring_uri_aks, input_data, headers=headers)
print(resp.text)

"{\"result\": [\"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Default Layer\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Layer 4\", \"Lay